<a href="https://colab.research.google.com/github/Almonfrey/MAI-Course/blob/main/class_8_practical_activity_scikit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

### Loading Data

In [ ]:
# Download data
url = "https://raw.githubusercontent.com/Almonfrey/MAI-Course/main/data/bank_marketing.csv"
data = pd.read_csv(url)

### Data Analysis

In [ ]:
print(f"Total of records: {len(data)}")
print("\nFirst rows of the dataset:")
data.head()

print("Data set information:")
data.info()

print("\nDistribution of the target 'deposit' variable:")
print(data['deposit'].value_counts())

print("\nDescriptive statistics:")
data.describe()

Total of records: 45211

First rows of the dataset:
Data set information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  deposit    45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB

Distribution of the targ

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


### Label Preprocess

In [ ]:
# --- Convert target to binary
data['deposit'] = data['deposit'].apply(lambda x: 1 if x == 2 else 0)

# --- Separate features and target
X = data.drop(columns=['deposit'])
y = data['deposit']

### Preprocess Pipeline
##### A brief remark: when defining a pipeline, you are not yet executing any operations. Instead, you are creating a recipe (or workflow) that will be executed later when you call the methods fit, predict, or evaluate.

In [ ]:
# --- Identify numeric and categorical columns
numeric_cols = X.select_dtypes(include=[np.number]).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# --- Define preprocessing for numerical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),   # fill missing with median
    ('to_float', FunctionTransformer(lambda x: x.astype('float32')))  # ensure float32 dtype
])

# --- Define preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # fill missing with mode
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))  # safe one-hot
])

# --- Combine preprocessing for both types
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop'  # drop any unlisted columns
)

### Modeling Pipeline
##### You are not yet executing any operation; instead, you are building a workflow that will be executed when you call the methods fit, predict, or evaluate.

In [ ]:
# --- Define full modeling pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', MLPClassifier(
        max_iter=500,
        random_state=42,
        early_stopping=True,          # enable early stopping
        validation_fraction=0.1,      # 10% of training data for validation
        n_iter_no_change=10,          # stop if no improvement after 10 epochs
        tol=1e-4,                     # tolerance for improvement
        verbose=1                  # enable progress logs for each model
    ))
])

### Setuping GridSearchCV
Before the explanation bellow, you should also be aware no processing is happening, but just the definition of the workflow.

### 🔹 GridSearchCV Overview

`GridSearchCV` is a **hyperparameter optimization** technique provided by the *scikit-learn* library.  
It systematically tests different combinations of specified hyperparameters to identify the configuration that yields the **best performance** according to a chosen evaluation metric (e.g., accuracy, F1-score).

Rather than manually trying multiple settings, `GridSearchCV` automates the process by:

1. Training a model for each combination of parameters defined in `param_grid`.  
2. Evaluating each model through **cross-validation**.  
3. Selecting the configuration that achieves the best average score.

The “CV” in `GridSearchCV` stands for **cross-validation**, which ensures that the model’s performance is not overly dependent on a single train/test split.

---

### 🔹 Parameter Grid Explanation

In your code, `param_grid` defines the **search space** for the `MLPClassifier` hyperparameters used inside a pipeline.  
Each key corresponds to a parameter name, and each value is a list of possible settings.  
The double underscore (`__`) notation (e.g., `'classifier__activation'`) refers to parameters **within a pipeline step** — here, the final step named `'classifier'`.

| Parameter | Description | Tested Values |
|------------|--------------|----------------|
| **`classifier__hidden_layer_sizes`** | Defines the architecture (number and size of hidden layers) in the MLP (Multi-Layer Perceptron). Each tuple represents one configuration — for example, `(64, 32)` means two hidden layers with 64 and 32 neurons, respectively. | `[(32,), (64,), (64, 32), (128, 64, 32)]` |
| **`classifier__activation`** | Specifies the activation function for neurons. It introduces non-linearity into the model. Common choices include: `'relu'` (rectified linear unit), `'tanh'`, and `'logistic'` (sigmoid). | `['relu', 'tanh', 'logistic']` |
| **`classifier__learning_rate_init`** | Sets the initial learning rate for the optimizer. A smaller value results in slower but more stable learning; a larger value speeds up convergence but may overshoot the optimum. | `[0.001, 0.01, 0.1]` |
| **`classifier__batch_size`** | Determines how many samples are processed before the model’s weights are updated. Smaller batch sizes provide more updates and may generalize better; larger batches are faster but can converge to suboptimal minima. | `[32, 64, 128]` |
| **`classifier__solver`** | Chooses the optimization algorithm used for training. Here, `'adam'` (adaptive moment estimation) is selected, a robust optimizer suitable for most neural network applications. | `['adam']` |

> 🧩 In total, the grid defines  
> \(4 x 3 x 3 x 3 x 1 = 108\) different hyperparameter combinations to test.

---

### 🔹 Cross-Validation Process (`StratifiedKFold(n_splits=5)`)

In your code, cross-validation is defined using a `StratifiedKFold` object with `n_splits=5`, which performs **5-fold cross-validation** while preserving the **class distribution** in each fold.

The process works as follows:

1. Split the training data into **five stratified folds**, each maintaining the original class proportions.  
2. For each hyperparameter combination:  
   - Train the model on **four** folds (80% of the data).  
   - Validate it on the **remaining** fold (20%).  
3. Repeat this process **five times**, ensuring every fold is used exactly once as the validation set.  
4. Compute the **average accuracy** across the five validation results.  
5. Select the hyperparameter combination with the **highest mean validation accuracy**.

Using stratification provides a **more reliable performance estimate**, especially when classes are imbalanced, as each fold reflects the true class distribution.

---

### 🔹 Other GridSearchCV Parameters

| Parameter | Purpose |
|------------|----------|
| **`estimator=pipeline`** | Specifies the model (in this case, the preprocessing pipeline + MLP classifier) to optimize. |
| **`param_grid=param_grid`** | Defines which hyperparameters to vary and the range of their possible values. |
| **`cv=stratified_cv`** | Uses 5-fold stratified cross-validation. |
| **`scoring='accuracy'`** | Uses classification accuracy as the metric to compare models. |
| **`verbose=2`** | Prints detailed progress logs during training. |
| **`n_jobs=-1`** | Utilizes all available CPU cores for parallel computation, speeding up the grid search. |

---

### 🔹 Process Summary

1. `GridSearchCV` clones the pipeline for each parameter combination.  
2. It trains each clone across 5 cross-validation folds.  
3. The average accuracy across folds is computed for each configuration.  
4. The best-performing combination is selected and stored in `grid_search.best_params_`.  
5. The final model (`grid_search.best_estimator_`) is retrained on the **entire training dataset** using the optimal parameters.

---

### ✅ Summary

> The `GridSearchCV` procedure performs an exhaustive search across a predefined hyperparameter grid, using 5-fold cross-validation to evaluate the performance of each parameter combination.  
> In this case, it optimizes the architecture, activation function, learning rate, and batch size of a neural network (`MLPClassifier`), ultimately selecting the configuration that achieves the highest mean accuracy across validation folds.


In [ ]:
# --- Define parameter grid for MLP
param_grid = {
    'classifier__hidden_layer_sizes': [(32,), (64,), (64, 32), (128, 64, 32)],  # layer configurations
    'classifier__activation': ['relu', 'tanh', 'logistic'],  # activation functions
    'classifier__learning_rate_init': [0.001, 0.01, 0.1],  # learning rates
    'classifier__batch_size': [32, 64, 128],  # batch sizes
    'classifier__solver': ['adam'],  # optimizer (adam recommended for most cases)
}

# --- Stratified cross-validation
stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# --- Define cross-validation grid search
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=stratified_cv,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

Split Data

In [ ]:
# --- Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


### Training process using Grid Search and Cross Validation
At this stage, the entire workflow defined previously is executed during the training process. The grid_search object applies this predefined pipeline to carry out all cross-validation procedures and evaluate every combination of hyperparameters.

In [ ]:
# --- Fit grid search
grid_search.fit(X_train, y_train)

# --- Display best parameters
print("\nBest Parameters:")
print(grid_search.best_params_)

# --- Save the best model object
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Iteration 1, loss = 0.00193069
Validation score: 1.000000
Iteration 2, loss = 0.00002436
Validation score: 1.000000
Iteration 3, loss = 0.00002123
Validation score: 1.000000
Iteration 4, loss = 0.00001854
Validation score: 1.000000
Iteration 5, loss = 0.00001590
Validation score: 1.000000
Iteration 6, loss = 0.00001335
Validation score: 1.000000
Iteration 7, loss = 0.00001105
Validation score: 1.000000
Iteration 8, loss = 0.00000915
Validation score: 1.000000
Iteration 9, loss = 0.00000773
Validation score: 1.000000
Iteration 10, loss = 0.00000678
Validation score: 1.000000
Iteration 11, loss = 0.00000615
Validation score: 1.000000
Iteration 12, loss = 0.00000568
Validation score: 1.000000
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.

Best Parameters:
{'classifier__activation': 'relu', 'classifier__batch_size': 32, 'classifier__hidden_layer_sizes': (32,), 'classifier__learning

### Retrain the best model on the entire training set before evaluation

In [ ]:
# --- Retrain best model on entire training data
best_model.fit(X_train, y_train)

Iteration 1, loss = 0.00193069
Validation score: 1.000000
Iteration 2, loss = 0.00002436
Validation score: 1.000000
Iteration 3, loss = 0.00002123
Validation score: 1.000000
Iteration 4, loss = 0.00001854
Validation score: 1.000000
Iteration 5, loss = 0.00001590
Validation score: 1.000000
Iteration 6, loss = 0.00001335
Validation score: 1.000000
Iteration 7, loss = 0.00001105
Validation score: 1.000000
Iteration 8, loss = 0.00000915
Validation score: 1.000000
Iteration 9, loss = 0.00000773
Validation score: 1.000000
Iteration 10, loss = 0.00000678
Validation score: 1.000000
Iteration 11, loss = 0.00000615
Validation score: 1.000000
Iteration 12, loss = 0.00000568
Validation score: 1.000000
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('to_float',
                                                                   FunctionTransformer(func=<function <lambda> at 0x7d74fd645080>))]),
                                                  Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object'))])),
                ('classifier',
                 MLPClassifier(batch_size=32, early_stopping=True,
                               hidden_layer_sizes=(32,), max_iter=500,
                               random_state=42, verbose=1))])

### Test the trained model

In [ ]:
# --- Evaluate on test data
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)

print(f"\nTest Accuracy (after retraining best model): {test_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Test Accuracy (after retraining best model): 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9043

    accuracy                           1.00      9043
   macro avg       1.00      1.00      1.00      9043
weighted avg       1.00      1.00      1.00      9043

